In [1]:
import string 

In [2]:
all_chars = string.printable
vocab_size = len(all_chars)
vocab_dict = dict((c, i) for (i, c) in enumerate(all_chars))

def str2ints(s, vocab_dict):
  return [vocab_dict[c] for c in s]

def ints2str(x, vocab_array):
  return "".join([vocab_array[i] for i in x])

In [3]:
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import tqdm

/home/tk/university/zemi/pytorch/.venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
class ShakespeareDataset(Dataset):
  def __init__(self, path, chunk_size=200):
    data = str2ints(open(path).read().strip(), vocab_dict)
    data = torch.tensor(data, dtype=torch.int64).split(chunk_size)

    if len(data[-1]) < chunk_size:
      data = data[: -1]
    
    self.data = data
    self.n_chunks = len(self.data)
  
  def __len__(self):
    return self.n_chunks

  def __getitem__(self, idx):
    return self.data[idx]

In [5]:
ds = ShakespeareDataset("../data/tinyshakespeare/input.txt", chunk_size=200)
loader = DataLoader(ds, batch_size=32, shuffle=True, num_workers=4)

In [6]:
class SequenceGenerationNet(nn.Module):
  def __init__(self, num_embeddings, embedding_dim=50, hidden_size=50, num_layers=1, dropout=0.2):
    super().__init__()
    self.emb = nn.Embedding(num_embeddings, embedding_dim)
    self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers,batch_first=True, dropout=dropout)
    self.linear = nn.Linear(hidden_size, num_embeddings)

  def forward(self, x, h0=None):
    x = self.emb(x)
    x, h = self.lstm(x, h0)
    x = self.linear(x)
    return x, h

In [7]:
def generate_seq(net, start_phrase="The King said", length=200, temperature=0.8, device="cpu"):
    net.eval()
    result = []

    start_tensor = torch.tensor(
      str2ints(start_phrase, vocab_dict),
      dtype=torch.int64
    ).to(device)

    x0 = start_tensor.unsqueeze(0)
    o, h = net(x0)
    out_dist = o[:, -1].view(-1).exp()
    top_i = torch.multinomial(out_dist, 1)[0]
    result.append(top_i)

    for i in range(length):
      inp =torch.tensor([[top_i]], dtype=torch.int64)
      inp = inp.to(device)
      o, h = net(inp, h)
      out_dist = o.view(-1).exp()
      top_i = torch.multinomial(out_dist, 1)[0]
      result.append(top_i)

    return start_phrase + ints2str(result, all_chars)

In [8]:
from statistics import mean

In [9]:
net = SequenceGenerationNet(vocab_size, 20, 50, num_layers=2, dropout=0.1)
net = net.to("cuda:0")
opt = optim.Adam(net.parameters())
loss_f = nn.CrossEntropyLoss()

for epoch in range(50):
  net.train()
  losses = []
  for data in tqdm.tqdm(loader):
    x = data[:, :-1]
    y = data[:, 1:]

    x = x.to("cuda:0")
    y = y.to("cuda:0")

    y_pred, _ = net(x)
    loss = loss_f(y_pred.view(-1, vocab_size), y.view(-1))
    net.zero_grad()
    loss.backward()
    opt.step()
    losses.append(loss.item())
  print(epoch, mean(losses))
  with torch.no_grad():
    print(generate_seq(net, device="cuda:0"))

100%|██████████| 175/175 [00:00<00:00, 221.72it/s]


0 3.488059459413801
The King saidTHt
 ae
 eue lrd ehwlcegh-r  otneIr hm;Aemrs,a,d, sprohy midD,dr'w h oydrr isoIa stemwU eM?aehdg:,l .vi I t!srE IB
t okY ou tdn ",isal t
p 
iramn ri ofloeba
dasresIevcr i e
ehR
sr
 tbghecoE  Qia i y .l


100%|██████████| 175/175 [00:00<00:00, 303.99it/s]


1 2.970314734322684
The King saids
noer t ane ie fa tupe the;
:a tot, o osghe;GIf mdane eone ta ees-u, yeo.
EE:
I
 o Neee shat wtlsahr
u the a;d'D
Wacroe siramd Enitgr fhtato wai bttrkdsasPre ia, bo masWdtlr wuae the kmid ea mue tiau,


100%|██████████| 175/175 [00:00<00:00, 309.38it/s]


...

47 1.6948352302823748
The King said look'd sinted coms-ing out, trick' reaver?

ROGKZAREM:
In were ary'd lies times uncoveonly netser! they dies I
homy; I lies waick; a genclast- the first our honasting to gloved show at the paricusble:


100%|██████████| 175/175 [00:00<00:00, 276.04it/s]


48 1.6920573547908238
The King said's
Tad, I now
And he have weave and sins of your ward a fell, you have whot a gatound? where faiding, he eme, I pieron an ensost
My lotcer say, Seart tell, and joinful
That delast for the more.

YORK:



100%|██████████| 175/175 [00:00<00:00, 278.68it/s]


49 1.6886347695759365
The King said 'tis fly by pompermer it.

BAPNAN:
Be a not swering thele,'l him have me, peade:
As bite thee thou hems I now for my pramp sweet them.
Is him; he humsure floof
of queen,
a' thou his justivings that th
